# Redes neuronales (ejercicio)

### **Importante: comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría.

## Parte 1: aplicación de redes neuronales a clasificación (análisis de sentimientos)

Se pide aplicar un modelo de redes neuronales al problema de decidir si una crítica de cine es positiva o negativa. Para ello volvemos a usar los datos de IMDB (Internet Movie Database) que vimos en el módulo 2 (modelo probabilístico).

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPClassifier`
* Keras con Tensorflow: usar `Sequential` y capas tipo `Dense` con la arquitectura adecuda.


Aunque ya hemos visto que los datos están disponibles en http://ai.stanford.edu/~amaas/data/sentiment/ , en este caso pedimos cargar los datos usando la utilidad `imdb`de Keras. Se puede consultar en el manual de Keras: https://keras.io/datasets/ Cargarlos con `imdb.load_data` y usar los datos cargados como punto de partida a este ejercicio (tanto para su aplicar scikit learn como para aplicar keras). Prestar atención al formato en el que se cargar, que no es el mismo que hamos visto hasta ahora.  

Los textos han de ser vectorizados para que se puedan ser procesados por una red. Para esto, tenemos varias alternativas, usar una de ellas:

* Vectorizando "manualmente", definiendo una función en python que lo haga.
* Vectorizadores de scikit learn (ya vistos)
* Herramientas de vectorización de keras: https://keras.io/preprocessing/text/

Mostrar algunas pruebas realizadas con distintas arquitecturas y/o hiperparámetros. No es necesario ser muy exhaustivo ni usar `GridSearchCV` en scikit learn ni el equivalente en keras. Tan solo mostrar alguna experimentación y ajuste manual.  

### Ejercicio
Empecemos por cargar los datos de las críticas de `imdb` utilizando Keras. El método `load_data` devuelve dos tuplas de arrays: uno con el conjunto de entrenamiento y otro con el conjunto de prueba. Cada tupla tiene los datos (*X_train_code*, *X_test_code*) que son una lista de índices, y su correspondiente clasificación (*y_train*, *y_test*) en 1 (positivo) y 0 (negativo). 

In [1]:
# Importar datos
import tensorflow as tf 
from tensorflow import keras
(X_train_code, y_train), (X_test_code, y_test) = keras.datasets.imdb.load_data()
X_train_code.shape, X_test_code.shape

((25000,), (25000,))

La lista de índices de los datos son palabras codificadas, es decir, cada índice es una palabra codificada, por tanto una crítica completa es el conjunto de todas los índices de las palabras. Como queremos trabajar con las críticas, vamos a decodificarlas, tanto el conjunto de entrenamiento como el de prueba. Para ello utilizamos el método `get_word_index` con el que obtenemos el diccionario de palabras y el índice correspondiente a cada una de ellas. 

In [2]:
# Decodificar
word_index = keras.datasets.imdb.get_word_index()
inverted_word_index = dict((i, word) for (word, i) in word_index.items())

X_train_decode = []
for critic in X_train_code:
    X_train_decode.append(" ".join([inverted_word_index.get(i, '') for i in critic]))

X_test_decode = []
for critic in X_test_code:
    X_test_decode.append(" ".join([inverted_word_index.get(i, '') for i in critic]))

Para procesar los textos que hemos decodificado es necesario vectorizarlos. Vamos a aplicar los vectorizadores de scikit learn que hemos visto en sesiones anteriores. Primero entrenamos el vectorizador y a continuación aplicamos la vectorización a ambos grupos, para obtener los conjuntos con los que vamos a trabajar.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train_decode)
X_train = vect.transform(X_train_decode)
X_test = vect.transform(X_test_decode)

Ahora que tenemos los datos, vamos a aplicar dos modelo de redes neuronales. Empecemos primero por el de scikit_learn `MLPClassifier` dentro del módulo `neural_network`, que permitirá usar las redes multicapa hacia adelante. Vamos a ver el rendimiento que obtenemos con los valores por defecto, una capa oculta y 100 unidades en esa capa, y con el método `lbfgs`, ya que no necesita tantos ajustes y con los datos que manejamos (25.000 ejemplos) se espera que funcione relativamente bien.

In [4]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver="lbfgs", max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

print("Rendimiento en entrenamiento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp.score(X_test, y_test)))

Rendimiento en entrenamiento: 1.00
Rendimiento en el conjunto de prueba: 0.86


El rendimiento obtenido no es muy bueno, de hecho se puede ver que sobre el conjunto de entrenamiento se produce algo de sobreajuste. Vamos a aplicar regularización, mediente el parámetro *alpha*, a ver si conseguimos que mejore algo el rendimiento.

In [5]:
mlp_a1 = MLPClassifier(solver="lbfgs", max_iter=1000, alpha=1, random_state=42)
mlp_a1.fit(X_train, y_train)
print("Rendimiento en entrenamiento: {:.2f}".format(mlp_a1.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp_a1.score(X_test, y_test)))

mlp_a05 = MLPClassifier(solver="lbfgs", max_iter=1000, alpha=0.05, random_state=42)
mlp_a05.fit(X_train, y_train)
print("Rendimiento en entrenamiento: {:.2f}".format(mlp_a05.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp_a05.score(X_test, y_test)))

Rendimiento en entrenamiento: 1.00
Rendimiento en el conjunto de prueba: 0.87
Rendimiento en entrenamiento: 1.00
Rendimiento en el conjunto de prueba: 0.86


A pesar de aplicar regularización, se obtienen resultados similares.<br><br>
Veamos ahora qué resultados obtenemos aplicando *Keras* con Tensorflow usando el módulo `Sequential` y capas tipo `Dense`. Cada unidad de estas capas recibe tantas conexiones como unidades en la capa anterior más un sesgo. Siguiendo el ejercicio visto, vamos a crear una red neuronal con la misma configuración:
- Dos capas tipo `Dense`, con 300 y 100 unidades, respectivamente, ambas con función de activación ReLU.
- Una última capa (salida) de 1 unidad, con función de activación `sigmoide`. Tiene una sola unidad porque al usar sigmoide, devolverá un valor entre 0 y 1.


In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

2022-01-05 09:31:11.599130: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Con el método `compile` especificamos la función de pérdida, coste (`loss`); el optimizador (`optimizer`) para buscar los pesos adecuados y la métrica (`metrics`) para medir el rendimiento del modelo.

In [7]:
model.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])

Ahora podemos entrenar el modelo mediante el método `fit` que recibe el el conjunto de entrenamiento, el número de *epochs* y un conjunto de validación para ir midiendo el rendimiento durante el proceso. Este último es opcional, pero vamos a utilizarlo para seguir el ejemplo visto en la sesión.<br>
Para obtener este conjunto de validación vamos a coger 8000 ejemplos para el conjunto de entrenamiento y otros 8000 para el de validación.

In [8]:
history = model.fit(X_train, y_train, epochs=30)

Epoch 1/30


/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 300), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


782/782 [==============================] - 5s 5ms/step - loss: 0.5631 - accuracy: 0.7222
Epoch 2/30
782/782 [==============================] - 3s 4ms/step - loss: 0.4482 - accuracy: 0.8098
Epoch 3/30
782/782 [==============================] - 3s 4ms/step - loss: 0.3865 - accuracy: 0.8382
Epoch 4/30
782/782 [==============================] - 3s 4ms/step - loss: 0.3545 - accuracy: 0.8530
Epoch 5/30
782/782 [==============================] - 3s 4ms/step - loss: 0.3245 - accuracy: 0.8654
Epoch 6/30
782/782 [==============================] - 3s 4ms/step - loss: 0.3056 - accuracy: 0.8764
Epoch 7/30
782/782 [==============================] - 3s 4ms/step - loss: 0.2846 - accuracy: 0.8824
Epoch 8/30
782/782 [==============================] - 3s 4ms/step - loss: 0.2695 - accuracy: 0.8913
Epoch 9/30
782/782 [==============================] - 3s 4ms/step - loss: 0.2596 - accuracy: 0.8950
Epoch 10/30
782/782 [==============================] - 3s 4ms/step - loss: 0.2449 - accuracy: 0.9014
Epoch 11/3

Y ahora veamos el rendimiento que obtenemos sobre el conjunto de pruebas, que no está mal.

In [9]:
lost, accuracy = model.evaluate(X_test, y_test)
print("Rendimiento en el conjunto de prueba: {:.3f}".format(accuracy))

782/782 [==============================] - 2s 3ms/step - loss: 0.3874 - accuracy: 0.8765
Rendimiento en el conjunto de prueba: 0.877


## Parte 2: aplicación de redes neuronales a regresión (predicción del precio de la vivienda)

Se pide aplicar un modelo de redes neuronales al problema de predecir precios de vivienda usando el conjunto de datos  `Boston house prices`. 

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPRegressor`
* Keras con Tensorflow: usar nuevamente `Sequential` y capas tipo `Dense` con la arquitectura adecuada.

El conjunto de datos se puede cargar usando tanto scikit learn (`sklearn.datasets.load_boston`) como keras (`keras.datasets.boston_housing`). 

Como en la parte 1, se pide mostrar algunas pruebas de los resultados obtenidos usando distintas arquitecturas y/o hiperparámetros. 


### Ejercicio
Siguiendo la dinámica del ejercicio anterior, vamos a cargar el conjunto de datos `Boston house prices` y a continuación aplicar un modelo de redes neuronales utilizando Scikit learn y Keras.<br><br>Empecemos cargando los datos, para ello vamos a utilizar Keras mediante el método `load_data` que nos va a devolver dos tuplas de arrays: uno con el conjunto de entrenamiento y otro con el conjunto de prueba. Cada tupla tiene los datos (*X_train_code*, *X_test_code*), y su correspondiente clasificación (*y_train*, *y_test*) que son números entre 10 y 50 que representan el precio de la vivienda en miles de dólares.

In [10]:
import tensorflow as tf 
from tensorflow import keras
(X_train, y_train), (X_test, y_test) = keras.datasets.boston_housing.load_data()
print(X_train.shape, X_test.shape)
X_train[0]

(404, 13) (102, 13)


array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

Viendo los datos, antes de aplicar los modelos, es necesario normalizarlos

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
print(X_train[0])

[0.01378163 0.         0.28152493 0.         0.31481481 0.49980635
 0.91452111 0.29719123 0.13043478 0.22753346 0.89361702 1.
 0.46881898]


Ahora que están normalizados, vamos a aplicar en primer lugar el sistema de scikit_learn `MLPRegressor` para hacer regresión. Vamos a ver el rendimiento que obtenemos con los valores por defecto (*alpha = 0.0001*).

In [12]:
from sklearn.neural_network import MLPRegressor
mlpr = MLPRegressor(max_iter=1000, random_state=42)
mlpr.fit(X_train, y_train)

print("Rendimiento en entrenamiento: {:.3f}".format(mlpr.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.3f}".format(mlpr.score(X_test, y_test)))

Rendimiento en entrenamiento: 0.712
Rendimiento en el conjunto de prueba: 0.682


Los rendimientos obtenidos en ambos conjuntos no son muy buenos. Probemos con distintos valores de *alpha* para ver si regularizando obtenemos mejores resultados.

In [13]:
mlpr_1 = MLPRegressor(max_iter=1000, random_state=42, alpha = 1)
mlpr_1.fit(X_train, y_train)
print("Rendimiento en entrenamiento: {:.3f}".format(mlpr_1.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.3f}".format(mlpr_1.score(X_test, y_test)))

mlpr_05 = MLPRegressor(max_iter=1000, random_state=42, alpha = 0.005)
mlpr_05.fit(X_train, y_train)
print("Rendimiento en entrenamiento: {:.3f}".format(mlpr_05.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.3f}".format(mlpr_05.score(X_test, y_test)))

Rendimiento en entrenamiento: 0.695
Rendimiento en el conjunto de prueba: 0.676
Rendimiento en entrenamiento: 0.713
Rendimiento en el conjunto de prueba: 0.685


El rendimiento obtenido es prácticamente el mismo, mejora por milésimas con *alpha = 0.005*, en cambio con *alpha = 1* empeora.<br><br>Ahora vamos a aplicar *Keras* con Tensorflow usando el módulo `Sequential` y capas tipo `Dense`. Vamos a crear una red neuronal con la siguiente configuración:
- Una capa inicial de tipo `Dense` con tantas entradas como número de características (13), con 100 unidades y función de activación ReLU.
- Una capa intermedia de tipo `Dense`, con 100 unidades y función de activación ReLU.
- Una última capa, la de salida, con 1 unidad y sin indicar función de activación, ya que por defecto es lineal

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(100, activation="relu", input_shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(1))

Especificamos la función de pérdida (`loss`), el optimizador (`optimizer`) para buscar los pesos adecuados y la métrica (`metrics`) para medir el rendimiento del modelo. En esta ocasión, al tratarse de un problema de regresión indicamos la función de pérdida error cuadrático medio (*mse, MeanSquaredError)*, como métrica error absoluto medio (*mae, MeanAbsoluteError*) y el optimizador *RMSprop*, que como se indica en la documentación: mantiene un promedio móvil (descontado) del cuadrado de gradientes y divide el gradiente por la raíz de este promedio.

In [15]:
model.compile(loss="mse", optimizer="rmsprop", metrics=["mae"])

Entrenamos y evaluamos

In [16]:
history = model.fit(X_train, y_train, epochs=30)
model.evaluate(X_test, y_test)

Epoch 1/30
13/13 [==============================] - 0s 1ms/step - loss: 551.4251 - mae: 21.5651
Epoch 2/30
13/13 [==============================] - 0s 1ms/step - loss: 444.4572 - mae: 18.8074
Epoch 3/30
13/13 [==============================] - 0s 1ms/step - loss: 311.1948 - mae: 14.8081
Epoch 4/30
13/13 [==============================] - 0s 1ms/step - loss: 195.4271 - mae: 10.9500
Epoch 5/30
13/13 [==============================] - 0s 1ms/step - loss: 134.4777 - mae: 8.9187
Epoch 6/30
13/13 [==============================] - 0s 1ms/step - loss: 110.5285 - mae: 8.0074
Epoch 7/30
13/13 [==============================] - 0s 1ms/step - loss: 94.5248 - mae: 7.3276
Epoch 8/30
13/13 [==============================] - 0s 1ms/step - loss: 80.7615 - mae: 6.6521
Epoch 9/30
13/13 [==============================] - 0s 1ms/step - loss: 68.8009 - mae: 6.0808
Epoch 10/30
13/13 [==============================] - 0s 1ms/step - loss: 60.2723 - mae: 5.5933
Epoch 11/30
13/13 [==============================

[23.727569580078125, 3.545335292816162]

Viendo las estadísticas, en cada *epoch* se ha ido reduciendo la pérdida.<br><br>Veamos ahora la predicción de un ejemplo cualquiera y comprobamos si ha acertado.

In [17]:
y_proba = model.predict(X_test[6:7])
print("Valor de la predicción: ", y_proba[0][0])
print("Valor real: ", y_test[6])

Valor de la predicción:  29.042421
Valor real:  31.2
